In [2]:
import numpy
import pandas as pd
import numpy as np
from pytz import utc, timezone
from datetime import datetime, timedelta


In [4]:
df = pd.read_csv("../../reports/original/report_08_11.csv")

In [6]:
df[df["Broker"] == "IF"]

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
5,08/11/23,06:44:25:868612,06:44:35:903860,NaN,CVLY,IF,Buy,5.0,NaN,NaN,...,21.50,21.86,21.50,21.86,317.0,322.0,Market,False,NaN,NaN
12,08/11/23,06:44:59:165109,06:45:09:182035,NaN,IMNM,IF,Buy,14.0,NaN,NaN,...,6.90,7.19,6.90,7.19,106.0,120.0,Market,False,NaN,NaN
19,08/11/23,06:45:30:769364,06:45:41:258976,NaN,EWTX,IF,Buy,15.0,NaN,NaN,...,6.72,6.80,6.80,6.90,8480.0,9490.0,Market,False,NaN,NaN
32,08/11/23,06:51:23:441242,06:51:31:940395,NaN,CVLY,IF,Sell,5.0,NaN,NaN,...,21.50,21.86,21.50,21.86,352.0,357.0,Market,False,NaN,NaN
39,08/11/23,06:51:53:292977,06:52:01:785815,NaN,IMNM,IF,Sell,14.0,NaN,NaN,...,6.91,7.19,6.91,7.19,205.0,219.0,Market,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,08/11/23,11:30:56:731560,11:31:05:144152,NaN,DTOC,IF,Sell,9.0,NaN,NaN,...,10.50,10.55,10.50,10.55,299.0,308.0,Market,False,NaN,NaN
1324,08/11/23,11:31:06:533036,11:31:15:016781,NaN,OLMA,IF,Sell,10.0,NaN,NaN,...,10.58,10.61,10.58,10.61,196285.0,196295.0,Market,False,NaN,NaN
1325,08/11/23,11:33:34:727245,11:33:45:162440,NaN,PCYG,IF,Buy,100.0,NaN,NaN,...,9.74,9.90,9.74,9.90,24733.0,24833.0,Market,False,NaN,NaN
1326,08/11/23,11:33:46:337548,11:33:56:227055,NaN,ALXO,IF,Buy,20.0,NaN,NaN,...,5.04,5.07,5.04,5.07,80205.0,80225.0,Market,False,NaN,NaN


In [ ]:
def convert_int64_utc_to_pst(int64):
    utc_datetime = datetime.utcfromtimestamp(float(int64) / 1000)
    now_aware = utc.localize(utc_datetime)
    pst = now_aware.astimezone(timezone("US/Pacific"))
    return pst.strftime("%H:%M:%S")


In [ ]:
df.head(10)

In [ ]:
%time
df.loc[df['Broker'] == "E2", 'Broker Executed'] = df.loc[
    df['Broker'] == "E2", 'Broker Executed'].apply(
    lambda int64_time: convert_int64_utc_to_pst(int64_time))

In [ ]:
df.loc[(df['Broker'] == "ET") | (df['Broker'] == "E2")]

In [ ]:
def vectorized_calculate_rounded_price(price_array):
    return np.round(np.round(price_array, 2) - price_array, 4)


def calculate_rounded_price(price):
    if price:
        price = float(price)
        return round(round(price, 2) - price, 4)
    return price

In [ ]:
%%timeit
df["Rounded Price - Price"] = vectorized_calculate_rounded_price(
    df["Price"].to_numpy(dtype = 'float64'))

In [ ]:
%%timeit
df.loc[:, "Rounded Price - Price"] = df.loc[:, "Price"].apply(calculate_rounded_price)


In [ ]:
price_idx = df.columns.get_loc("Price")
df.insert(price_idx + 2, "PriceImprovement", 0)

In [ ]:
def calculate_tradeID(row):
    bigTrade = row["BigTrade"]
    symbol = row["Symbol"]
    today = datetime.strptime(row["Date"], "%x")
    year, month, day = today.year, today.strftime("%m"), today.strftime("%d")
    return f"{year}{month}{day}{symbol}{'1000' if bigTrade else '0100'}"

In [ ]:
post_vol_idx = df.columns.get_loc("Post Volume")
df.insert(post_vol_idx + 1, "First", 0)
df.insert(post_vol_idx + 2, "BigTrade", 0)
df.insert(post_vol_idx + 3, "TradeID", 0)


In [ ]:
%%timeit
df.loc[:, "TradeID"] = df.loc[:, ["Date", "Symbol", "BigTrade"]].apply(calculate_tradeID,
                                                                       axis = 1)

In [ ]:
vfunc = np.vectorize(calculate_tradeID)

In [ ]:
%%timeit

df["New Col"] = df[["Date", "Symbol", "BigTrade"]].apply(calculate_tradeID, axis = 1)

In [ ]:
df

In [ ]:
df["DateObj"] = pd.to_datetime(df["Date"])

In [ ]:
%%timeit

df["DateObj"].dt.year.astype(str) + df["DateObj"].dt.month.map("{:02}".format).astype(str) + df[
    "DateObj"].dt.day.astype(str) + df["Symbol"] + np.where(
    df["BigTrade"].to_numpy(dtype = "int64") == 0, "0100", "100")

In [ ]:
%%timeit


In [ ]:
%%timeit
np.where(df["BigTrade"] == 0, "0100", "100")

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df["Date"] = df["Date"].dt.strftime('%m/%d/%Y')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
split_idx = df.columns.get_loc("Split")
df.insert(split_idx + 1, "BidAskSpread", 0)
df.insert(split_idx + 2, "TradeLocation", 0)

In [ ]:
type(np.nan)

In [ ]:
float(np.nan)

In [ ]:
df["BidAskSpread"] = (df["Pre Ask"] - df["Pre Bid"]).round(4)


In [ ]:
df

In [ ]:
program_submitted_idx = df.columns.get_loc("Program Submitted")
df.insert(program_submitted_idx + 1, "Program Submitted Text", "")

program_executed_idx = df.columns.get_loc("Program Executed")
df.insert(program_executed_idx + 1, "Program Executed Text", "")

broker_executed_idx = df.columns.get_loc("Broker Executed")
df.insert(broker_executed_idx + 1, "Broker Executed Text", "")

In [ ]:
df

In [ ]:
df["Program Submitted"] = pd.to_datetime(df["Program Submitted"], format = '%X:%f')
df["Program Executed"] = pd.to_datetime(df["Program Executed"], format = '%X:%f')

In [ ]:
pd.to_datetime(df["Broker Executed"])

# Fidelity Stuff

In [ ]:
file1 = '/Users/sanathnair/Developer/trading/brokers/unopened.txt'
dfs1 = pd.read_html(file1)

In [ ]:
del dfs1[1]

In [ ]:
dfs1 = dfs1[0]

In [ ]:
dfs1 = dfs1.drop(dfs1.columns[[0, 2, 4]], axis = 1)

In [ ]:
dfs1.rename(columns = {1: 'Info', 3: "Price"}, inplace = True)

In [ ]:
dfs1

In [ ]:
dfs1["Price"] = dfs1["Price"].str.slice(start = 11).astype("float64")
dfs1

In [ ]:
dfs1_temp = dfs1["Info"].str.split(expand = True)
dfs1 = dfs1.join(dfs1_temp)

In [ ]:
dfs1.rename(
    columns = {0: 'Action', 1: "Quantity", 2: "Shares", 3: "of", 4: "Symbol", 5: "at", 6: "type",
               7: "when"}, inplace = True)

In [ ]:
dfs1 = dfs1.drop(["Shares", "of", "at", "type", "when", "Info"], axis = 1)

In [ ]:
dfs1

## dfs2 - split trades

In [ ]:
file2 = '/Users/sanathnair/Developer/trading/brokers/opened.txt'
dfs2 = pd.read_html(file2)

In [ ]:
dfs2

In [ ]:
del dfs2[-1]

In [ ]:
del dfs2[0]

In [ ]:
dfs2

In [ ]:
dfs2[0]

In [ ]:
np.ones((6, 1))

In [ ]:
splits = dfs2[0].iloc[:-1].to_numpy()
length = splits.shape[0]
identifier = np.empty((length, 1))
identifier.fill(1)
test = np.hstack((splits, identifier))
test2 = np.hstack((splits, identifier))
# np.append(test, test2, axis = 0)

In [ ]:
# get amount for each trade
prices = []
for idx, temp in enumerate(dfs2):
    splits = temp.iloc[:-1].to_numpy()
    length = splits.shape[0]
    identifier = np.empty((length, 1))
    identifier.fill(idx)
    updated = np.hstack((splits, identifier))
    prices.append(updated)
prices
# prices = pd.Series(prices, dtype = "float64")

In [ ]:
res = prices[0]
for x in prices[1:]:
    res = np.append(res, x, axis = 0)

res.shape
    

In [ ]:
splits_df = pd.DataFrame(res, columns = ["Date", "Broker Executed", "Price", "Quantity", "Dollar Amt", "Identifier" ])


In [ ]:
merged = pd.merge(splits_df, dfs1[['Symbol', 'Action']], left_on='Identifier', right_index=True)


In [ ]:
merged["Broker Executed"] = merged["Broker Executed"].str.slice(stop=-3)

In [ ]:
merged["Broker Executed"] = pd.to_datetime(merged["Broker Executed"], format = '%I:%M:%S %p', utc = False)

In [ ]:
merged["Price"] = merged["Price"].str.slice(start=1)

In [ ]:
merged["Dollar Amt"] = merged["Dollar Amt"].str.slice(start=1)

In [ ]:
merged["Broker Executed"] = merged["Broker Executed"] + pd.Timedelta(hours = -3)

In [ ]:
merged.to_csv("split_info.csv", index = False)

In [ ]:
merged = merged[::-1]

In [ ]:
merged

# How to combine them?

In [144]:
import pandas as pd

In [146]:
report_file = pd.read_csv("../../reports/original/report_08_17.csv")
splits = pd.read_csv("../../data/fidelity_splits/fd_splits_08_17.csv")

In [147]:
report_file[(report_file["Broker"] == "FD")]

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
2,08/17/23,06:45:24:422772,06:45:29:939180,NaN,BAC,FD,Buy,3.0,NaN,NaN,...,29.40,29.41,29.41,29.42,2145338.0,2147586.0,Market,False,NaN,NaN
7,08/17/23,06:45:52:280427,06:45:57:759303,NaN,DISH,FD,Buy,14.0,NaN,NaN,...,7.18,7.19,7.19,7.20,355954.0,360640.0,Market,False,NaN,NaN
12,08/17/23,06:46:17:410329,06:46:22:874263,NaN,PFMT,FD,Buy,42.0,NaN,NaN,...,2.42,2.48,2.44,2.48,2839.0,2881.0,Market,False,NaN,NaN
17,08/17/23,06:46:42:399716,06:46:47:902116,NaN,NNBR,FD,Buy,46.0,NaN,NaN,...,2.18,2.24,2.18,2.24,2572.0,2618.0,Market,False,NaN,NaN
23,08/17/23,06:50:46:423156,06:50:51:908253,NaN,BAC,FD,Sell,3.0,NaN,NaN,...,29.34,29.35,29.34,29.35,2557717.0,2559226.0,Market,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,08/17/23,09:56:45:183109,09:56:50:651451,NaN,GWRE,FD,Buy,1.0,NaN,NaN,...,80.52,80.57,80.52,80.57,133711.0,133712.0,Market,False,NaN,NaN
563,08/17/23,10:01:03:746457,10:01:09:229196,NaN,SIX,FD,Sell,5.0,NaN,NaN,...,21.29,21.30,21.29,21.30,304013.0,304018.0,Market,False,NaN,NaN
568,08/17/23,10:01:35:933614,10:01:41:424139,NaN,GS,FD,Sell,1.0,NaN,NaN,...,332.66,332.82,332.59,332.80,1093387.0,1094062.0,Market,False,NaN,NaN
573,08/17/23,10:02:07:968121,10:02:13:411533,NaN,PXLW,FD,Sell,78.0,NaN,NaN,...,1.29,1.30,1.29,1.30,229827.0,229905.0,Market,False,NaN,NaN


In [183]:
splits

,Date,Broker Executed,Price,Quantity,Dollar Amt,Identifier,Symbol,Action
0,08/17/2023,1900-01-01 10:02:45,80.4200,1.0,80.42,0.0,GWRE,Sell
1,08/17/2023,1900-01-01 10:02:12,1.3000,78.0,101.40,1.0,PXLW,Sell
2,08/17/2023,1900-01-01 10:01:40,332.8000,1.0,332.80,2.0,GS,Sell
3,08/17/2023,1900-01-01 10:01:08,21.2950,5.0,106.48,3.0,SIX,Sell
4,08/17/2023,1900-01-01 09:56:50,80.5565,1.0,80.56,4.0,GWRE,Buy
...,...,...,...,...,...,...,...,...
103,08/17/2023,1900-01-01 06:50:51,29.3500,3.0,88.05,101.0,BAC,Sell
104,08/17/2023,1900-01-01 06:46:47,2.2127,46.0,101.78,102.0,NNBR,Buy
105,08/17/2023,1900-01-01 06:46:22,2.4600,42.0,103.32,103.0,PFMT,Buy
106,08/17/2023,1900-01-01 06:45:57,7.1933,14.0,100.71,104.0,DISH,Buy


In [200]:
def parse_fidelity_splits(row):
    symbol = row["Symbol"]
    action = row["Action"]
    report_row = report_file[(report_file["Broker"] == "FD") & (report_file["Symbol"] == symbol) & (report_file["Action"] == action)]
    if report_row.shape[0]:
        report_row["Broker Executed"] = row["Broker Executed"]
        report_row["Size"] = row["Quantity"]
        report_row["Price"] = row["Price"]
        report_row["Dollar Amt"] = row["Dollar Amt"]
        return report_row.squeeze()
    # print(report_row.to_numpy())

In [217]:
temp = splits.apply(parse_fidelity_splits, axis = 1)
temp[temp["Date"].notna()]

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,08/17/23,10:02:40:004490,10:02:45:436190,1900-01-01 10:02:45,GWRE,FD,Sell,1.0,80.4200,80.42,...,80.41,80.49,80.41,80.48,139880.0,139983.0,Market,False,NaN,NaN
1,08/17/23,10:02:07:968121,10:02:13:411533,1900-01-01 10:02:12,PXLW,FD,Sell,78.0,1.3000,101.40,...,1.29,1.30,1.29,1.30,229827.0,229905.0,Market,False,NaN,NaN
2,08/17/23,10:01:35:933614,10:01:41:424139,1900-01-01 10:01:40,GS,FD,Sell,1.0,332.8000,332.80,...,332.66,332.82,332.59,332.80,1093387.0,1094062.0,Market,False,NaN,NaN
3,08/17/23,10:01:03:746457,10:01:09:229196,1900-01-01 10:01:08,SIX,FD,Sell,5.0,21.2950,106.48,...,21.29,21.30,21.29,21.30,304013.0,304018.0,Market,False,NaN,NaN
4,08/17/23,09:56:45:183109,09:56:50:651451,1900-01-01 09:56:50,GWRE,FD,Buy,1.0,80.5565,80.56,...,80.52,80.57,80.52,80.57,133711.0,133712.0,Market,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,08/17/23,06:50:46:423156,06:50:51:908253,1900-01-01 06:50:51,BAC,FD,Sell,3.0,29.3500,88.05,...,29.34,29.35,29.34,29.35,2557717.0,2559226.0,Market,False,NaN,NaN
104,08/17/23,06:46:42:399716,06:46:47:902116,1900-01-01 06:46:47,NNBR,FD,Buy,46.0,2.2127,101.78,...,2.18,2.24,2.18,2.24,2572.0,2618.0,Market,False,NaN,NaN
105,08/17/23,06:46:17:410329,06:46:22:874263,1900-01-01 06:46:22,PFMT,FD,Buy,42.0,2.4600,103.32,...,2.42,2.48,2.44,2.48,2839.0,2881.0,Market,False,NaN,NaN
106,08/17/23,06:45:52:280427,06:45:57:759303,1900-01-01 06:45:57,DISH,FD,Buy,14.0,7.1933,100.71,...,7.18,7.19,7.19,7.20,355954.0,360640.0,Market,False,NaN,NaN


In [204]:
# %%timeit
pd.concat([report_file, splits.apply(parse_fidelity_splits, axis = 1)])
# updated.reset_index()

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,08/17/23,06:45:05:922167,06:45:15:898579,NaN,BAC,IF,Buy,3.0,NaN,NaN,...,29.39,29.40,29.41,29.42,2127408.0,2141617.0,Market,False,NaN,NaN
1,08/17/23,06:45:16:856921,06:45:17:019760,06:45:17,BAC,TD,Buy,3.0,29.4099,88.2297,...,29.41,29.42,29.40,29.41,2141617.0,2142767.0,Market,False,11484295175,5.872606e+10
2,08/17/23,06:45:24:422772,06:45:29:939180,NaN,BAC,FD,Buy,3.0,NaN,NaN,...,29.40,29.41,29.41,29.42,2145338.0,2147586.0,Market,False,NaN,NaN
3,08/17/23,06:45:31:387475,06:45:32:515823,NaN,BAC,RH,Buy,3.0,NaN,NaN,...,29.41,29.42,29.41,29.42,2147377.0,2147377.0,Limit,False,64de247c-2820-4c8e-b4e6-b7edcbf19acb,NaN
4,08/17/23,06:45:33:458208,06:45:34:541989,1692279934545,BAC,E2,Buy,3.0,NaN,NaN,...,29.41,29.42,29.41,29.42,681234.0,681238.0,Market,False,3596,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,08/17/23,06:50:46:423156,06:50:51:908253,1900-01-01 06:50:51,BAC,FD,Sell,3.0,29.3500,88.0500,...,29.34,29.35,29.34,29.35,2557717.0,2559226.0,Market,False,NaN,NaN
104,08/17/23,06:46:42:399716,06:46:47:902116,1900-01-01 06:46:47,NNBR,FD,Buy,46.0,2.2127,101.7800,...,2.18,2.24,2.18,2.24,2572.0,2618.0,Market,False,NaN,NaN
105,08/17/23,06:46:17:410329,06:46:22:874263,1900-01-01 06:46:22,PFMT,FD,Buy,42.0,2.4600,103.3200,...,2.42,2.48,2.44,2.48,2839.0,2881.0,Market,False,NaN,NaN
106,08/17/23,06:45:52:280427,06:45:57:759303,1900-01-01 06:45:57,DISH,FD,Buy,14.0,7.1933,100.7100,...,7.18,7.19,7.19,7.20,355954.0,360640.0,Market,False,NaN,NaN


In [152]:
pd.concat((report_file, updated))

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
0,08/17/23,06:45:05:922167,06:45:15:898579,NaN,BAC,IF,Buy,3.0,NaN,NaN,...,29.39,29.40,29.41,29.42,2127408.0,2141617.0,Market,False,NaN,NaN
1,08/17/23,06:45:16:856921,06:45:17:019760,06:45:17,BAC,TD,Buy,3.0,29.4099,88.2297,...,29.41,29.42,29.40,29.41,2141617.0,2142767.0,Market,False,11484295175,5.872606e+10
2,08/17/23,06:45:24:422772,06:45:29:939180,NaN,BAC,FD,Buy,3.0,NaN,NaN,...,29.40,29.41,29.41,29.42,2145338.0,2147586.0,Market,False,NaN,NaN
3,08/17/23,06:45:31:387475,06:45:32:515823,NaN,BAC,RH,Buy,3.0,NaN,NaN,...,29.41,29.42,29.41,29.42,2147377.0,2147377.0,Limit,False,64de247c-2820-4c8e-b4e6-b7edcbf19acb,NaN
4,08/17/23,06:45:33:458208,06:45:34:541989,1692279934545,BAC,E2,Buy,3.0,NaN,NaN,...,29.41,29.42,29.41,29.42,681234.0,681238.0,Market,False,3596,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,08/17/23,06:50:46:423156,06:50:51:908253,1900-01-01 06:50:51,BAC,FD,Sell,3.0,29.3500,88.0500,...,29.34,29.35,29.34,29.35,2557717.0,2559226.0,Market,False,NaN,NaN
17,08/17/23,06:46:42:399716,06:46:47:902116,1900-01-01 06:46:47,NNBR,FD,Buy,46.0,2.2127,101.7800,...,2.18,2.24,2.18,2.24,2572.0,2618.0,Market,False,NaN,NaN
12,08/17/23,06:46:17:410329,06:46:22:874263,1900-01-01 06:46:22,PFMT,FD,Buy,42.0,2.4600,103.3200,...,2.42,2.48,2.44,2.48,2839.0,2881.0,Market,False,NaN,NaN
7,08/17/23,06:45:52:280427,06:45:57:759303,1900-01-01 06:45:57,DISH,FD,Buy,14.0,7.1933,100.7100,...,7.18,7.19,7.19,7.20,355954.0,360640.0,Market,False,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np

In [250]:
df = pd.read_html('/Users/sanathnair/Developer/trading/data/ibkr_prices/original/DailyTradeReport.20230816.html')

In [251]:
df[1]

,Acct ID,Symbol,Symbol.1,Trade Date/Time,Settle Date,Exchange,Type,Quantity,Price,Proceeds,Comm,Fee,Order Type,Code
0,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks,Stocks
1,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
2,U***5820,AAPL,AAPL,"2023-08-16, 11:41:03",2023-08-18,-,BUY,1,177.7871,-177.79,0.00,0.00,MKT,O
3,Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),1,177.7871,-177.79,0.00,0.00,NaN,NaN
4,U***5820,AAPL,AAPL,"2023-08-16, 11:47:56",2023-08-18,-,SELL,-1,177.5713,177.57,0.00¹,0.00,MKT,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,U***5820,ZUMZ,ZUMZ,"2023-08-16, 14:07:30",2023-08-18,-,SELL,-6,17.9929,107.96,0.00¹,0.00,MKT,C
478,Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),-6,17.9929,107.96,0.00,0.00,NaN,NaN
479,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,0,NaN,0.13,0.00,0.00,NaN,NaN
480,Total,Total,Total,Total,Total,Total,Total,Total,Total,-49.13,-0.31,0.00,NaN,NaN


In [252]:
df = df[1]

In [253]:
df.drop(index = df.index[:2], inplace = True)

In [254]:
df

,Acct ID,Symbol,Symbol.1,Trade Date/Time,Settle Date,Exchange,Type,Quantity,Price,Proceeds,Comm,Fee,Order Type,Code
2,U***5820,AAPL,AAPL,"2023-08-16, 11:41:03",2023-08-18,-,BUY,1,177.7871,-177.79,0.00,0.00,MKT,O
3,Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),Total AAPL (Bought),1,177.7871,-177.79,0.00,0.00,NaN,NaN
4,U***5820,AAPL,AAPL,"2023-08-16, 11:47:56",2023-08-18,-,SELL,-1,177.5713,177.57,0.00¹,0.00,MKT,C
5,Total AAPL (Sold),Total AAPL (Sold),Total AAPL (Sold),Total AAPL (Sold),Total AAPL (Sold),Total AAPL (Sold),Total AAPL (Sold),-1,177.5713,177.57,0.00,0.00,NaN,NaN
6,Total AAPL,Total AAPL,Total AAPL,Total AAPL,Total AAPL,Total AAPL,Total AAPL,0,NaN,-0.22,0.00,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,U***5820,ZUMZ,ZUMZ,"2023-08-16, 14:07:30",2023-08-18,-,SELL,-6,17.9929,107.96,0.00¹,0.00,MKT,C
478,Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),Total ZUMZ (Sold),-6,17.9929,107.96,0.00,0.00,NaN,NaN
479,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,Total ZUMZ,0,NaN,0.13,0.00,0.00,NaN,NaN
480,Total,Total,Total,Total,Total,Total,Total,Total,Total,-49.13,-0.31,0.00,NaN,NaN


In [255]:
df_sub = df[["Trade Date/Time", "Symbol", "Quantity", "Price", "Code"]]

In [256]:
df_sub = df_sub[df_sub["Code"].notna()]

In [257]:
df_sub["Code"] = df_sub["Code"].str[0]
df_sub["Action"] = np.where(df_sub["Code"] == "O", "Buy", "Sell")

In [258]:
df_sub = df_sub.drop("Code", axis=1)

In [259]:
df_sub = df_sub.drop(df.index[-1])

In [260]:
df_sub

,Trade Date/Time,Symbol,Quantity,Price,Action
2,"2023-08-16, 11:41:03",AAPL,1,177.7871,Buy
4,"2023-08-16, 11:47:56",AAPL,-1,177.5713,Sell
7,"2023-08-16, 12:40:36",AGL,6,17.9099,Buy
9,"2023-08-16, 12:47:30",AGL,-6,17.9000,Sell
12,"2023-08-16, 11:51:04",AGX,2,40.5715,Buy
...,...,...,...,...,...
467,"2023-08-16, 12:36:59",WTFC,-1,79.4100,Sell
470,"2023-08-16, 12:31:05",XOM,1,107.8900,Buy
472,"2023-08-16, 12:38:02",XOM,-1,107.8511,Sell
475,"2023-08-16, 14:00:33",ZUMZ,6,17.9713,Buy


In [261]:
df_sub["Trade Date/Time"] = pd.to_datetime(df_sub["Trade Date/Time"])

In [262]:
df_sub["Trade Date/Time"] = df_sub["Trade Date/Time"] - pd.Timedelta(hours = 3)

In [263]:
df_sub = df_sub.set_axis(["Time", "Symbol", "Quantity", "Price", "Action"], axis=1)

In [264]:
df_sub["Quantity"] = df_sub["Quantity"].astype("int64").abs()
df_sub["Price"] = df_sub["Price"].astype("float64")

In [265]:
df_sub

,Time,Symbol,Quantity,Price,Action
2,2023-08-16 08:41:03,AAPL,1,177.7871,Buy
4,2023-08-16 08:47:56,AAPL,1,177.5713,Sell
7,2023-08-16 09:40:36,AGL,6,17.9099,Buy
9,2023-08-16 09:47:30,AGL,6,17.9000,Sell
12,2023-08-16 08:51:04,AGX,2,40.5715,Buy
...,...,...,...,...,...
467,2023-08-16 09:36:59,WTFC,1,79.4100,Sell
470,2023-08-16 09:31:05,XOM,1,107.8900,Buy
472,2023-08-16 09:38:02,XOM,1,107.8511,Sell
475,2023-08-16 11:00:33,ZUMZ,6,17.9713,Buy


In [266]:
df_sub["Dollar Amt"] = (df_sub["Price"] * df_sub["Quantity"]).abs().round(2)

In [280]:
from datetime import datetime
datetime.now().strftime("%I")

'01'

In [267]:
df_sub

,Time,Symbol,Quantity,Price,Action,Dollar Amt
2,2023-08-16 08:41:03,AAPL,1,177.7871,Buy,177.79
4,2023-08-16 08:47:56,AAPL,1,177.5713,Sell,177.57
7,2023-08-16 09:40:36,AGL,6,17.9099,Buy,107.46
9,2023-08-16 09:47:30,AGL,6,17.9000,Sell,107.40
12,2023-08-16 08:51:04,AGX,2,40.5715,Buy,81.14
...,...,...,...,...,...,...
467,2023-08-16 09:36:59,WTFC,1,79.4100,Sell,79.41
470,2023-08-16 09:31:05,XOM,1,107.8900,Buy,107.89
472,2023-08-16 09:38:02,XOM,1,107.8511,Sell,107.85
475,2023-08-16 11:00:33,ZUMZ,6,17.9713,Buy,107.83


In [270]:
report_file = pd.read_csv("../../reports/original/report_08_16.csv")

In [281]:
def parse_ibkr_prices(row):
    symbol = row["Symbol"]
    action = row["Action"]
    hour = row["Time"].strftime("%I")
    report_row = report_file[
        (report_file["Broker"] == "IF") & (report_file["Symbol"] == symbol) & (report_file["Action"] == action) & (report_file["Program Submitted"].str[:2] == hour)]
    if report_row.shape[0]:
        report_row["Broker Executed"] = row["Time"].strftime("%X")
        report_row["Size"] = row["Quantity"]
        report_row["Price"] = row["Price"]
        report_row["Dollar Amt"] = row["Dollar Amt"]
        return report_row.squeeze()

In [272]:
report_file[report_file["Broker"] == "IF"]

,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
3,08/16/23,06:44:08:849930,06:44:18:874821,NaN,EWTX,IF,Buy,14.0,NaN,NaN,...,7.21,7.33,7.21,7.33,3698.0,3769.0,Market,False,NaN,NaN
9,08/16/23,06:44:39:123782,06:44:50:638951,NaN,V,IF,Buy,1.0,NaN,NaN,...,240.32,240.41,240.23,240.41,387994.0,388986.0,Market,False,NaN,NaN
15,08/16/23,06:45:11:897586,06:45:22:379024,NaN,EBIX,IF,Buy,7.0,NaN,NaN,...,14.82,14.91,14.82,14.91,27938.0,28054.0,Market,False,NaN,NaN
21,08/16/23,06:45:41:879957,06:45:51:811835,NaN,INZY,IF,Buy,21.0,NaN,NaN,...,4.76,4.78,4.76,4.78,21721.0,21742.0,Market,False,NaN,NaN
27,08/16/23,06:51:08:981925,06:51:18:527639,NaN,EWTX,IF,Sell,14.0,NaN,NaN,...,7.23,7.36,7.23,7.36,4189.0,4204.0,Market,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,08/16/23,11:26:52:820827,11:27:03:127544,NaN,V,IF,Sell,1.0,NaN,NaN,...,240.24,240.31,240.24,240.33,1887753.0,1888901.0,Market,False,NaN,NaN
1288,08/16/23,11:27:23:644722,11:27:33:177056,NaN,EBIX,IF,Sell,7.0,NaN,NaN,...,14.63,14.70,14.63,14.70,383293.0,383300.0,Market,False,NaN,NaN
1294,08/16/23,11:27:53:518767,11:28:02:942708,NaN,INZY,IF,Sell,21.0,NaN,NaN,...,4.75,4.77,4.75,4.77,347953.0,347974.0,Market,False,NaN,NaN
1295,08/16/23,11:50:53:236776,11:51:04:063792,NaN,OLMA,IF,Sell,10.0,NaN,NaN,...,9.79,9.88,9.79,9.88,184227.0,184237.0,Market,False,NaN,NaN


In [282]:
df_sub.apply(parse_ibkr_prices, axis = 1)


,Date,Program Submitted,Program Executed,Broker Executed,Symbol,Broker,Action,Size,Price,Dollar Amt,...,Pre Bid,Pre Ask,Post Bid,Post Ask,Pre Volume,Post Volume,Order Type,Split,Order ID,Activity ID
2,08/16/23,08:40:55:143445,08:41:05:052946,08:41:03,AAPL,IF,Buy,1.0,177.7871,177.79,...,177.78,177.79,177.80,177.81,17359108.0,17365416.0,Market,False,NaN,NaN
4,08/16/23,08:47:48:534703,08:47:57:960925,08:47:56,AAPL,IF,Sell,1.0,177.5713,177.57,...,177.58,177.59,177.57,177.58,17973062.0,17983302.0,Market,False,NaN,NaN
7,08/16/23,09:40:26:156404,09:40:37:688806,09:40:36,AGL,IF,Buy,6.0,17.9099,107.46,...,17.89,17.91,17.89,17.91,375280.0,375294.0,Market,False,NaN,NaN
9,08/16/23,09:47:22:976362,09:47:32:494074,09:47:30,AGL,IF,Sell,6.0,17.9000,107.40,...,17.89,17.91,17.89,17.91,382504.0,382511.0,Market,False,NaN,NaN
12,08/16/23,08:50:55:417567,08:51:05:275632,08:51:04,AGX,IF,Buy,2.0,40.5715,81.14,...,40.49,40.59,40.49,40.59,8215.0,8218.0,Market,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,08/16/23,09:36:52:235395,09:37:01:810486,09:36:59,WTFC,IF,Sell,1.0,79.4100,79.41,...,79.41,79.49,79.39,79.43,49291.0,49429.0,Market,False,NaN,NaN
470,08/16/23,09:30:56:669704,09:31:06:578142,09:31:05,XOM,IF,Buy,1.0,107.8900,107.89,...,107.87,107.88,107.88,107.89,4924254.0,4925683.0,Market,False,NaN,NaN
472,08/16/23,09:37:54:463146,09:38:04:015598,09:38:02,XOM,IF,Sell,1.0,107.8511,107.85,...,107.85,107.86,107.86,107.87,5030673.0,5032876.0,Market,False,NaN,NaN
475,08/16/23,11:00:25:216130,11:00:35:160797,11:00:33,ZUMZ,IF,Buy,6.0,17.9713,107.83,...,17.96,17.98,17.96,17.98,230579.0,230611.0,Market,False,NaN,NaN


In [283]:
pd.read_csv("../../data/fidelity_splits/fd_splits_08_17.csv")

,Date,Broker Executed,Price,Quantity,Dollar Amt,Identifier,Symbol,Action
0,08/17/2023,1900-01-01 10:02:45,80.4200,1.0,80.42,0.0,GWRE,Sell
1,08/17/2023,1900-01-01 10:02:12,1.3000,78.0,101.40,1.0,PXLW,Sell
2,08/17/2023,1900-01-01 10:01:40,332.8000,1.0,332.80,2.0,GS,Sell
3,08/17/2023,1900-01-01 10:01:08,21.2950,5.0,106.48,3.0,SIX,Sell
4,08/17/2023,1900-01-01 09:56:50,80.5565,1.0,80.56,4.0,GWRE,Buy
...,...,...,...,...,...,...,...,...
103,08/17/2023,1900-01-01 06:50:51,29.3500,3.0,88.05,101.0,BAC,Sell
104,08/17/2023,1900-01-01 06:46:47,2.2127,46.0,101.78,102.0,NNBR,Buy
105,08/17/2023,1900-01-01 06:46:22,2.4600,42.0,103.32,103.0,PFMT,Buy
106,08/17/2023,1900-01-01 06:45:57,7.1933,14.0,100.71,104.0,DISH,Buy


In [1]:
import pandas as pd